# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl, ref_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    label = sp.io.loadmat(ref_fl)['BPM0']
    
    return data[2:], label

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl, gbm)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl, gbm):
    # Load data using LoadTroikaDataFile
    labels, subjects, features, labels_pred, confidences = [], [], [], [], []
    data, label = LoadTroikaDataFile(data_fl, ref_fl)
    dataset = pd.DataFrame({'PPG': data[0], 'accx': data[1], 'accy': data[2], 'accz': data[3]})
    dataset['acc_total'] = np.sqrt(data[1]**2 + data[2]**2 + data[3]**2)
    window_length_s = 8
    window_shift_s = 2
    window_length = window_length_s * fs
    window_shift = window_shift_s * fs
    checkers = []
    for j in range(0, len(dataset) - window_length, window_shift):
        window = dataset[j: j + window_length]
        ppg = window.PPG.values
        accx = window.accx.values
        accy = window.accy.values
        accz = window.accz.values
        acc = window.acc_total.values
        info = Featurize(ppg, accx, accy, accz, fs)
        checkers.append(info)
        features.append(info)
        pred = gbm.predict([info])
        labels_pred.append(pred[0])
        ppg_fft, ppg_mag, freqs = process_ppg(ppg, 40, 240, 125)
        acc_fft, acc_mag = process_acc(acc, 40, 240, 125)
        window_f = 5 / 60
        fund_freq_window = (freqs > pred/60 - window_f) & \
                            (freqs < pred/60 + window_f)
        ppg_power = np.sum(ppg_fft[fund_freq_window])
        total_power = np.sum(ppg_mag)
        confidence = ppg_power / total_power
        confidences.append(confidence)
        
    for k in label[:len(checkers)]:
        labels.append(k[0])
    errors = np.abs(np.asarray(labels)-np.asarray(labels_pred))
    # Compute pulse rate estimates and estimation confidence.

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    #errors, confidence = np.ones(100), np.ones(100)  # Dummy placeholders. Remove
    return errors, confidences

def BandpassFilter(signal, fs):
    b, a = sp.signal.butter(3, (0.6667, 4.0), btype='bandpass', fs=fs)#(3, 12, btype='lowpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def Featurize(ppg, accx, accy, accz, fs):
    ppg = BandpassFilter(ppg, fs)
    accx = BandpassFilter(accx, fs)
    accy = BandpassFilter(accy, fs)
    accz = BandpassFilter(accz, fs)

    # The mean of each channel
    mn_ppg = np.mean(ppg)
    mn_x = np.mean(accx)
    mn_y = np.mean(accy)
    mn_z = np.mean(accz)

    # The standard deviation of each channel
    std_ppg = np.std(ppg)
    std_x = np.std(accx)
    std_y = np.std(accy)
    std_z = np.std(accz)

    # Various percentile values for each channel
    p5_ppg = np.percentile(ppg, 5)
    p5_x = np.percentile(accx, 5)
    p5_y = np.percentile(accy, 5)
    p5_z = np.percentile(accz, 5)
    p10_ppg = np.percentile(ppg, 10)
    p10_x = np.percentile(accx, 10)
    p10_y = np.percentile(accy, 10)
    p10_z = np.percentile(accz, 10)
    p25_ppg = np.percentile(ppg, 25)
    p25_x = np.percentile(accx, 25)
    p25_y = np.percentile(accy, 25)
    p25_z = np.percentile(accz, 25)
    p50_ppg = np.percentile(ppg, 50)
    p50_x = np.percentile(accx, 50)
    p50_y = np.percentile(accy, 50)
    p50_z = np.percentile(accz, 50)
    p90_ppg = np.percentile(ppg, 90)
    p90_x = np.percentile(accx, 90)
    p90_y = np.percentile(accy, 90)
    p90_z = np.percentile(accz, 90)

    # The pearson correlation of all pairs of channels
    corr_xy = sp.stats.pearsonr(accx, accy)[0]
    corr_xz = sp.stats.pearsonr(accx, accz)[0]
    corr_yz = sp.stats.pearsonr(accy, accz)[0]

    # The total energy of each channel
    energy_ppg = np.sum(np.square(ppg - np.mean(ppg)))
    energy_x = np.sum(np.square(accx - np.mean(accx)))
    energy_y = np.sum(np.square(accy - np.mean(accy)))
    energy_z = np.sum(np.square(accz - np.mean(accz)))

    # Take an FFT of the signal. If the signal is too short, 0-pad it so we have 
    # at least 2046 points in the FFT.
    n = len(ppg) * 4
    fft_len = max(len(accx), n)

    # Create an array of frequency bins
    fft_freqs = np.fft.rfftfreq(fft_len, 1 / fs)

    # Helper function to select frequency bins between <low> and <high>
    freqs_bw = lambda low, high: (fft_freqs >= low) & (fft_freqs <= high)

    # Compute the accelerometer magnitude
    accm = np.sqrt(np.sum(np.square(np.vstack((accx, accy, accz))), axis=0))

    # Take an FFT of the centered signal
    fft_ppg = np.fft.rfft(accx, n)
    fft_ppg[fft_freqs <= 40/60.0] = 0.0
    fft_ppg[fft_freqs >= 240/60.0] = 0.0
    fft_x = np.fft.rfft(accx, n)
    fft_y = np.fft.rfft(accy, n)
    fft_z = np.fft.rfft(accz, n)
    fft_m = np.fft.rfft(accm, n)

    # Compute the energy spectrum
    spec_energy_ppg = np.square(np.abs(fft_ppg))
    spec_energy_x = np.square(np.abs(fft_x))
    spec_energy_y = np.square(np.abs(fft_y))
    spec_energy_z = np.square(np.abs(fft_z))
    spec_energy_m = np.square(np.abs(fft_m))

    # The frequency with the most power between 0.25 and 12 Hz
    dom_ppg = fft_freqs[np.argmax(fft_ppg[freqs_bw(0.25, 12)])]
    dom_x = fft_freqs[np.argmax(fft_x[freqs_bw(0.25, 12)])]
    dom_y = fft_freqs[np.argmax(fft_y[freqs_bw(0.25, 12)])]
    dom_z = fft_freqs[np.argmax(fft_z[freqs_bw(0.25, 12)])]
    dom_m = fft_freqs[np.argmax(fft_m[freqs_bw(0.25, 12)])]

    # The fraction of energy in various frequency bins for each channel
    energy_01_ppg = (np.sum(spec_energy_ppg[freqs_bw(0, 1)]) / np.sum(spec_energy_ppg))
    energy_01_x = (np.sum(spec_energy_x[freqs_bw(0, 1)]) / np.sum(spec_energy_x))
    energy_01_y = (np.sum(spec_energy_x[freqs_bw(0, 1)]) / np.sum(spec_energy_y))
    energy_01_z = (np.sum(spec_energy_x[freqs_bw(0, 1)]) / np.sum(spec_energy_z))
    energy_01_m = (np.sum(spec_energy_x[freqs_bw(0, 1)]) / np.sum(spec_energy_m))
    energy_12_ppg = (np.sum(spec_energy_ppg[freqs_bw(1, 2)]) / np.sum(spec_energy_ppg))
    energy_12_x = (np.sum(spec_energy_x[freqs_bw(1, 2)]) / np.sum(spec_energy_x))
    energy_12_y = (np.sum(spec_energy_x[freqs_bw(1, 2)]) / np.sum(spec_energy_y))
    energy_12_z = (np.sum(spec_energy_x[freqs_bw(1, 2)]) / np.sum(spec_energy_z))
    energy_12_m = (np.sum(spec_energy_x[freqs_bw(1, 2)]) / np.sum(spec_energy_m))
    energy_23_ppg = (np.sum(spec_energy_ppg[freqs_bw(2, 3)]) / np.sum(spec_energy_ppg))
    energy_23_x = (np.sum(spec_energy_x[freqs_bw(2, 3)]) / np.sum(spec_energy_x))
    energy_23_y = (np.sum(spec_energy_x[freqs_bw(2, 3)]) / np.sum(spec_energy_y))
    energy_23_z = (np.sum(spec_energy_x[freqs_bw(2, 3)]) / np.sum(spec_energy_z))
    energy_23_m = (np.sum(spec_energy_x[freqs_bw(2, 3)]) / np.sum(spec_energy_m))
    energy_34_ppg = (np.sum(spec_energy_ppg[freqs_bw(3, 4)]) / np.sum(spec_energy_ppg))
    energy_34_x = (np.sum(spec_energy_x[freqs_bw(3, 4)]) / np.sum(spec_energy_x))
    energy_34_y = (np.sum(spec_energy_x[freqs_bw(3, 4)]) / np.sum(spec_energy_y))
    energy_34_z = (np.sum(spec_energy_x[freqs_bw(3, 4)]) / np.sum(spec_energy_z))
    energy_34_m = (np.sum(spec_energy_x[freqs_bw(3, 4)]) / np.sum(spec_energy_m))
    energy_45_ppg = (np.sum(spec_energy_ppg[freqs_bw(4, 5)]) / np.sum(spec_energy_ppg))
    energy_45_x = (np.sum(spec_energy_x[freqs_bw(4, 5)]) / np.sum(spec_energy_x))
    energy_45_y = (np.sum(spec_energy_x[freqs_bw(4, 5)]) / np.sum(spec_energy_y))
    energy_45_z = (np.sum(spec_energy_x[freqs_bw(4, 5)]) / np.sum(spec_energy_z))
    energy_45_m = (np.sum(spec_energy_x[freqs_bw(4, 5)]) / np.sum(spec_energy_m))
    energy_56_ppg = (np.sum(spec_energy_ppg[freqs_bw(5, 6)]) / np.sum(spec_energy_ppg))
    energy_56_x = (np.sum(spec_energy_x[freqs_bw(5, 6)]) / np.sum(spec_energy_x))
    energy_56_y = (np.sum(spec_energy_x[freqs_bw(5, 6)]) / np.sum(spec_energy_y))
    energy_56_z = (np.sum(spec_energy_x[freqs_bw(5, 6)]) / np.sum(spec_energy_z))
    energy_56_m = (np.sum(spec_energy_x[freqs_bw(5, 6)]) / np.sum(spec_energy_m))
  

    return [mn_ppg,
            mn_x,
            mn_y,
            mn_z,
            std_ppg,
            std_x,
            std_y,
            std_z,
            p5_ppg,
            p5_x,
            p5_y,
            p5_z,
            p10_ppg,
            p10_x,
            p10_y,
            p10_z,
            p25_ppg,
            p25_x,
            p25_y,
            p25_z,
            p50_ppg,
            p50_x,
            p50_y,
            p50_z,
            p90_ppg,
            p90_x,
            p90_y,
            p90_z,
            corr_xy,
            corr_xz,
            corr_yz,
            energy_ppg,
            energy_x,
            energy_y,
            energy_z,
            dom_ppg,
            dom_x,
            dom_y,
            dom_z,
            dom_m,
            energy_01_ppg,
            energy_12_ppg,
            energy_23_ppg,
            energy_34_ppg,
            energy_45_ppg,
            energy_56_ppg,
            energy_01_x,
            energy_12_x,
            energy_23_x,
            energy_34_x,
            energy_45_x,
            energy_56_x,
            energy_01_y,
            energy_12_y,
            energy_23_y,
            energy_34_y,
            energy_45_y,
            energy_56_y,
            energy_01_z,
            energy_12_z,
            energy_23_z,
            energy_34_z,
            energy_45_z,
            energy_56_z,
            energy_01_m,
            energy_12_m,
            energy_23_m,
            energy_34_m,
            energy_45_m,
            energy_56_m,
           ]

def process_ppg(ppg, low_bpm, high_bpm, fs):
    """
    Process the PPG Signal
    
    Args:
        ppg: PPG Signal
        freqs: a numpy array of frequency bins
        ftt_len: length of ppg
        low_bpm: minimum bpm 
        high_bpm: maximum bpm
    Returns:
        Processed PPG Signal and Max PPG Frequency
    
    """
    ppg = BandpassFilter(ppg, fs)
    ftt_len = max(len(ppg), 2046)
    freqs = np.fft.rfftfreq(ftt_len, 1 / fs)
    
    ppg_fft = np.abs(np.fft.rfft(ppg, ftt_len))
    pks = sp.signal.find_peaks(ppg_fft, height=2000, distance=1500)[0]
    ppg_mag = freqs[np.argsort(ppg_fft)[-3:][::-1]]
    
    return ppg_fft, ppg_mag, freqs

def process_acc(acc, low_bpm, high_bpm, fs):
    """
    Process the ACC Signal
    
    Args:
        accx: (np.array) x-channel of the accelerometer.
        accy: (np.array) y-channel of the accelerometer.
        accz: (np.array) z-channel of the accelerometer.
        freqs: a numpy array of frequency bins
        ftt_len: length of acc
        low_bpm: minimum bpm 
        high_bpm: maximum bpm
    Returns:
        Processed ACC Signal and Max ACC Frequency
    """
    
    ftt_len = max(len(acc), 2046)
    freqs = np.fft.rfftfreq(ftt_len, 1 / fs)
    
    acc = BandpassFilter(acc, fs)
    
    acc_fft = np.abs(np.fft.rfft(acc, ftt_len))
    acc_mag = freqs[np.argmax(acc_fft)]
    
    return acc_fft, acc_mag

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
fs =125
%matplotlib inline
data_fls, ref_fls = LoadTroikaDataset()
labels, subjects, features = [], [], []
for i in range(len(data_fls)):
    data, label = LoadTroikaDataFile(data_fls[i], ref_fls[i])
    dataset = pd.DataFrame({'PPG': data[0], 'accx': data[1], 'accy': data[2], 'accz': data[3]})
    window_length_s = 8
    window_shift_s = 2
    window_length = window_length_s * fs
    window_shift = window_shift_s * fs
    checkers = []
    for j in range(0, len(dataset) - window_length, window_shift):
        window = dataset[j: j + window_length]
        ppg = window.PPG.values
        accx = window.accx.values
        accy = window.accy.values
        accz = window.accz.values
        info = Featurize(ppg, accx, accy, accz, fs)
        checkers.append(info)
        features.append(info)
    for k in label[:len(checkers)]:
        labels.append(k[0])
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
gbm = GradientBoostingRegressor(n_estimators=150, max_depth=10, loss='ls', criterion='mae', max_features=15, min_samples_split=6, learning_rate = 0.06)
gbm.fit(X_train, y_train)

Evaluate()

0.98736466344458063